# ROL - WaveBot


In [1]:
import autograd.numpy as np
import capytaine as cpy
import matplotlib.pyplot as plt

import wecopttool as wot

plt.style.use('tableau-colorblind10')

### Waves

In [2]:
wavefreq = 0.3 # Hz
f1 = wavefreq
nfreq = 10
freq = wot.frequency(f1, nfreq, False)

amplitude = 0.0625 # m
phase = 30 # degrees
wavedir = 0 # degrees

waves = wot.waves.regular_wave(f1, nfreq, wavefreq, amplitude, phase, wavedir)

### WEC object

In [3]:
wb = wot.geom.WaveBot()
mesh_size_factor = 0.5
mesh = wb.mesh(mesh_size_factor)
fb = cpy.FloatingBody.from_meshio(mesh, name="WaveBot")

fb.add_translation_dof(name="Heave")
ndof = fb.nb_dofs

bem_data = wot.run_bem(fb, freq)

name = ["PTO_Heave",]
kinematics = np.eye(ndof)
controller = None
loss = None
pto_impedance = None
pto = wot.pto.PTO(ndof, kinematics, controller, pto_impedance, loss, name)

f_add = {'PTO': pto.force_on_wec}

wec = wot.WEC.from_bem(bem_data, f_add=f_add)

[13:14:06] WARNING  Using the geometric centroid as the center of gravity (COG).

           WARNING  Using the center of gravity (COG) as the rotation center for hydrostatics.

The resolution of the mesh might be insufficient for omega ranging from 9.425 to 18.850.
This warning appears when the largest panel of this mesh has radius > wavelength/8.


           WARNING  Mesh resolution for 12 problems:                                                               
                    The resolution of the mesh might be insufficient for omega ranging from 9.425 to 18.850.       
                    This warning appears when the largest panel of this mesh has radius > wavelength/8.

Output()

[13:14:09] WARNING  Linear damping for DOF "Heave" has negative or close to zero terms. Shifting up damping terms  
                    [5 7 8 9] to a minimum of 1e-06 N/(m/s)

### Objective function

In [4]:
obj_fun = pto.mechanical_average_power
nstate_opt = 2*nfreq

### Solve

In [5]:
results = wec.solve(
    waves,
    obj_fun,
    nstate_opt,
    )

opt_mechanical_average_power = results[0]['f']
print(f'Optimal average mechanical power: {opt_mechanical_average_power} W')

/Users/cmichel/repos/WecOptTool/wecopttool/core.py:673: UserWarning: The `squeeze` kwarg to GroupBy is being removed.Pass .groupby(..., squeeze=False) to disable squeezing, which is the new default, and to silence this warning.
  for realization, wave in waves.groupby('realization'):



  Check primal optimization space vector

********** Begin verification of linear algebra. *********************************************

Commutativity of addition. Consistency error: >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> 0.000000000000e+00
Associativity of addition. Consistency error: >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> 6.718913446119e-16
Identity element of addition. Consistency error: >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> 0.000000000000e+00
Inverse elements of addition. Consistency error: >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> 0.000000000000e+00
Identity element of scalar multiplication. Consistency error: >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> 0.000000000000e+00
Consistency of scalar multiplication with field multiplication. Consistency error: >>>>>>>>>>> 3.239802184843e-16
Distributivity of scalar multiplication with respect to field addition. Consistency error: >>> 5.059686135104e-16
Distributivity of scalar multiplication with respect to vector 

### Analyzing results

In [6]:
results

[{'x': array([-2.16798125e-05,  1.66246828e-01, -2.25297848e-01, -1.33222976e-12,
          5.35288716e-13,  7.10398479e-10,  5.99716482e-11, -1.20361476e-07,
          4.12181580e-07, -8.08319844e-07, -4.84792579e-06, -2.95984858e-06,
          3.28245550e-06,  2.82538917e-06, -4.62195628e-06, -5.07458223e-06,
         -1.20754087e-06,  1.59237512e-07,  1.09504184e-06, -2.01271214e-06,
         -5.28836260e-01,  2.36280187e+03, -4.09546786e+03, -5.17067703e-09,
         -7.27611051e-09, -1.56892527e-05,  3.75122956e-07,  7.23291555e-03,
         -2.78359909e-02,  1.02235572e-01,  5.95740395e-01,  5.65844786e-01,
         -6.27518699e-01, -7.67762434e-01,  1.25630419e+00,  1.84795186e+00,
          4.39736178e-01, -7.46595857e-02, -5.13417782e-01,  1.17905071e+00]),
  'f': -139.98091445983306}]